In [3]:
import pandas as pd
import h3
import keplergl as KeplerGl

In [6]:
brd_charging_df = pd.read_csv('/Users/dk/Visualcode/Capstone/capstone_project/data/brd.csv')

In [48]:
muc_charging_df = brd_charging_df.query('Ort == "München"')

In [50]:
muc_charging_df.columns = muc_charging_df.columns.str.replace(' ', '_')
muc_charging_df.columns = muc_charging_df.columns.str.lower()
muc_charging_df.columns = muc_charging_df.columns.str.rstrip()


In [52]:
muc_charging_df['breitengrad']=muc_charging_df['breitengrad'].apply(pd.to_numeric)
muc_charging_df['längengrad']=muc_charging_df['längengrad'].apply(pd.to_numeric)
# df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)

<ipython-input-52-663903972dbe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muc_charging_df['breitengrad']=muc_charging_df['breitengrad'].apply(pd.to_numeric)
<ipython-input-52-663903972dbe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muc_charging_df['längengrad']=muc_charging_df['längengrad'].apply(pd.to_numeric)


In [53]:
def make_h3_code(row):
    return h3.geo_to_h3(
            lat=row['breitengrad'],
            lng=row['längengrad'],
            resolution=8    # Average Hexagon Area (km2): 0.7373276
                            # Average Hexagon Edge Length (km)
                            # https://h3geo.org/docs/core-library/restable/
)

In [54]:
muc_charging_df['h3_code'] = muc_charging_df.apply(lambda row : make_h3_code(row), axis=1)

<ipython-input-54-0ea912f1d864>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muc_charging_df['h3_code'] = muc_charging_df.apply(lambda row : make_h3_code(row), axis=1)


In [55]:
muc_charging_df = muc_charging_df[[ 'betreiber',
                                    'postleitzahl',
                                    'breitengrad',
                                    'längengrad',
                                    'inbetriebnahmedatum',
                                    'h3_code']]

In [56]:
muc_charging_df.head()

,betreiber,postleitzahl,breitengrad,längengrad,inbetriebnahmedatum,h3_code
4502,SWM Versorgungs GmbH,80241,48.137018,11.456796,31/1/2020,881f8d4515fffff
4503,SWM Versorgungs GmbH,80245,48.159746,11.445709,7/12/2018,881f8d4551fffff
4504,SWM Versorgungs GmbH,80245,48.159701,11.445643,7/12/2018,881f8d4551fffff
4505,SWM Versorgungs GmbH,80249,48.147492,11.419531,11/12/2018,881f8d4547fffff
4506,SWM Versorgungs GmbH,80331,48.135646,11.580395,21/12/2017,881f8d7a49fffff


In [57]:
muc_charging_df.h3_code.nunique()

204

In [58]:
muc_charging_df.to_csv('muc_charging_df.csv', encoding='utf-8', index=False)

In [ ]:
muc_charging_groupby_h3 = muc_charging_df.groupby('h3_code')